# 图像（数据）增广
- Data Augmentation

**核心问题：训练时的数据和实际使用时的数据长得不一样！**

### 图像增广就是提前模拟各种情况

```
你的训练数据：都是在办公室拍的，光线好，角度正

实际使用时：
    可能光线暗
    可能颜色偏黄
    可能只拍到物体的一半
    可能图片是翻转的

图像增广 = 把训练数据变换一下，模拟这些情况
    让模型提前见过各种"奇怪"的图片
    这样实际使用时就不会懵了
```
---

## 二、常用的增广方法（三种最重要）

### 方法1：翻转

```
左右翻转：最常用，几乎所有任务都用
    猫朝左 → 翻转 → 猫朝右
    一张图变两张图！

上下翻转：看情况用
    猫翻过来头朝下 → 很奇怪 → 一般不用
    树叶、卫星图 → 可以用（没有上下之分）
```

### 方法2：随机裁剪

```
从图片中随机切一块出来，再缩放到固定大小

为什么？
    实际拍照时，物体可能只占画面一部分
    可能只拍到猫的头
    可能只拍到猫的尾巴
    模型应该都能认出来

具体做法：
    随机选位置
    随机选大小（占原图8%~100%）
    随机选高宽比（3/4 到 4/3）
    切出来后缩放到224×224
```

### 方法3：颜色变换

```
随机改变亮度、色调、饱和度

为什么？
    不同灯光下颜色不同（白天vs晚上）
    不同相机拍出来颜色不同
    模型不应该因为颜色变了就认不出来

具体做法：
    亮度：在0.5到1.5之间随机（变暗或变亮）
    色调：随机偏红、偏蓝、偏黄
    饱和度：颜色浓一点或淡一点
```

---

## 三、什么时候用？

```
训练时：每次读图片都随机做增广
        同一张猫的图片，每次训练看到的都不一样
        → 相当于增加了数据量

测试/推理时：不做增广！用原始图片
        → 要稳定的结果，不能加随机性
```

**这也是一种正则化方法（和Dropout类似，只在训练时用）**

---

## 代码实现

### 定义增广策略
```python
import torchvision.transforms as transforms

# 训练时的数据变换（有增广）
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # 随机左右翻转
    transforms.RandomResizedCrop(224), # 随机裁剪到224×224
    transforms.ColorJitter( # 颜色变换
        brightness=0.5,    # 亮度：±50%
        contrast=0.5,   # 对比度：±50%
        saturation=0.5,    # 饱和度：±50%
        hue=0.1  色调：±10%
    ),
    transforms.ToTensor(),   # 转成Tensor
])

# 测试时的数据变换（没有增广！）
test_transform = transforms.Compose([
    transforms.Resize(256),  # 缩放到256
    transforms.CenterCrop(224),   # 从中心裁224×224
    transforms.ToTensor(),   # 转成Tensor
])
```

### 逐行解释

```python
transforms.RandomHorizontalFlip()
```

```
随机左右翻转
有50%概率翻转，50%概率不翻
每次读图片都重新随机
```

```python
transforms.RandomResizedCrop(224)
```

```
随机裁剪一块，然后缩放到224×224
位置随机、大小随机、高宽比随机
```

```python
transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)
```

```
随机改变颜色
brightness=0.5 → 亮度在50%~150%之间随机
contrast=0.5   → 对比度在50%~150%之间随机
saturation=0.5 → 饱和度在50%~150%之间随机
hue=0.1        → 色调轻微变化
```

```python
transforms.ToTensor()
```

```
把图片转成PyTorch的Tensor格式
这个必须写，不管有没有增广
```

### 测试时为什么不同？

```python
# 测试时：
transforms.Resize(256)  # 先缩放到256（固定的，不随机）
transforms.CenterCrop(224)# 从正中间裁224（固定的，不随机）
transforms.ToTensor()

# 测试时不能随机！
# 因为你想要稳定的预测结果
# 同一张图片预测两次，结果应该一样
```

---

## 五、怎么用在训练里？

```python
from torchvision import datasets

# 加载数据时，指定transform
train_dataset = datasets.ImageFolder(
    'data/train', 
    transform=train_transform # 训练用带增广的transform
)

test_dataset = datasets.ImageFolder(
    'data/test', 
    transform=test_transform # 测试用不带增广的transform
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
```

**就是在加载数据的时候指定transform，其他代码完全不变！**

---

## 六、增广的效果

```
实际效果：
    准确率通常能提高2%~5%
    在数据量小的时候提升更明显
```

---

## 七、总结

### 需要记住的

```
1. 图像增广 = 训练时随机变换图片，模拟各种情况
2. 三种最常用：翻转、裁剪、颜色变换
3. 只在训练时用，测试时不用！
4. 代码上就是定义transform，加载数据时传进去
```

### 代码模板

```python
# 训练transform（有随机增广）
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, 
                           saturation=0.5, hue=0.1),
    transforms.ToTensor(),
])

# 测试transform（没有随机，固定操作）
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
```
